<a href="https://colab.research.google.com/github/samkaj/appliedml-1/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

By Group 5: Samuel Kajava and Torbjörn Livén

[Instructions](https://www.cse.chalmers.se/~richajo/dit866/assignments/a1/assignment1.html) here.

In [2]:
# Fetch the CTG data set.
!wget https://www.cse.chalmers.se/~richajo/dit866/data/CTG.csv

--2024-01-15 16:37:33--  https://www.cse.chalmers.se/~richajo/dit866/data/CTG.csv
Resolving www.cse.chalmers.se (www.cse.chalmers.se)... 129.16.221.33
Connecting to www.cse.chalmers.se (www.cse.chalmers.se)|129.16.221.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 307385 (300K) [text/plain]
Saving to: ‘CTG.csv’

CTG.csv             100%[===================>] 300,18K  --.-KB/s    in 0,04s   

2024-01-15 16:37:33 (6,80 MB/s) - ‘CTG.csv’ saved [307385/307385]



In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the CSV file.
data = pd.read_csv('CTG.csv', skiprows=1)

# Select the relevant numerical columns.
selected_cols = ['LB', 'AC', 'FM', 'UC', 'DL', 'DS', 'DP', 'ASTV', 'MSTV', 'ALTV',
                 'MLTV', 'Width', 'Min', 'Max', 'Nmax', 'Nzeros', 'Mode', 'Mean',
                 'Median', 'Variance', 'Tendency', 'NSP']
data = data[selected_cols].dropna()

# Shuffle the dataset.
data_shuffled = data.sample(frac=1.0, random_state=0)

# Split into input part X and output part Y.
X = data_shuffled.drop('NSP', axis=1)

# Map the diagnosis code to a human-readable label.
def to_label(y):
    return [None, 'normal', 'suspect', 'pathologic'][(int(y))]

Y = data_shuffled['NSP'].apply(to_label)

# Partition the data into training and test sets.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)

In [4]:
# Take a peak at the data.
X.head()

,LB,AC,FM,UC,DL,DS,DP,ASTV,MSTV,ALTV,...,Width,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency
658,130.0,1.0,0.0,3.0,0.0,0.0,0.0,24.0,1.2,12.0,...,35.0,120.0,155.0,1.0,0.0,134.0,133.0,135.0,1.0,0.0
1734,134.0,9.0,1.0,8.0,5.0,0.0,0.0,59.0,1.2,0.0,...,109.0,80.0,189.0,6.0,0.0,150.0,146.0,150.0,33.0,0.0
1226,125.0,1.0,0.0,4.0,0.0,0.0,0.0,43.0,0.7,31.0,...,21.0,120.0,141.0,0.0,0.0,131.0,130.0,132.0,1.0,0.0
1808,143.0,0.0,0.0,1.0,0.0,0.0,0.0,69.0,0.3,6.0,...,27.0,132.0,159.0,1.0,0.0,145.0,144.0,146.0,1.0,0.0
825,152.0,0.0,0.0,4.0,0.0,0.0,0.0,62.0,0.4,59.0,...,25.0,136.0,161.0,0.0,0.0,159.0,156.0,158.0,1.0,1.0


## Step 2. Training the baseline classifier

We begin by using a dummy classifier as a baseline for our upcomping implementation. A higher aggregated result means that the accuracy is higher, so we want to find a classifier with a good aggregated result.

In [5]:
from sklearn.dummy import DummyClassifier

# Create dummy classifier.
clf = DummyClassifier(strategy='most_frequent')

In [8]:
from sklearn.model_selection import cross_val_score

# Perform cross validation.
dummy_cross_val = cross_val_score(clf, Xtrain, Ytrain)

Once we have cross validated the scores, we now aggregate the results to make it comparable to other classifiers.

In [11]:
import numpy as np

# Returns the aggregation of an array of numbers.
def aggregate(arr: np.ndarray):
  return sum(arr)

dummy_aggr = aggregate(dummy_cross_val)
dummy_aggr

3.902941176470588

## Step 3. Trying out different classifiers

In this step, we use a number of classifiers and compare the aggregated results using the `aggregate()` function defined above.

In [13]:
import sklearn.tree as tree
import sklearn.ensemble as ensemble
import sklearn.linear_model as linear
import sklearn.svm as svm 
import sklearn.neural_network as nn

# Run cross validation on decision tree, random forest, gradient boosting, perceptron, logistic regression, linear SVC and MLP.
clf = tree.DecisionTreeClassifier()
tree_cross_val = cross_val_score(clf, Xtrain, Ytrain)

clf = ensemble.RandomForestClassifier()
forest_cross_val = cross_val_score(clf, Xtrain, Ytrain)

clf = ensemble.GradientBoostingClassifier()
boost_cross_val = cross_val_score(clf, Xtrain, Ytrain)

clf = linear.Perceptron(max_iter=10000)
perceptron_cross_val = cross_val_score(clf, Xtrain, Ytrain)

clf = linear.LogisticRegression(max_iter=10000)
logreg_cross_val = cross_val_score(clf, Xtrain, Ytrain)

clf = svm.LinearSVC(max_iter=10000)
svc_cross_val = cross_val_score(clf, Xtrain, Ytrain)

clf = nn.MLPClassifier()
mlp_cross_val = cross_val_score(clf, Xtrain, Ytrain)

# Aggregate the results.
tree_aggr = aggregate(tree_cross_val)
forest_aggr = aggregate(forest_cross_val)
boost_aggr = aggregate(boost_cross_val)
perceptron_aggr = aggregate(perceptron_cross_val)
logreg_aggr = aggregate(logreg_cross_val)
svc_aggr = aggregate(svc_cross_val)
mlp_aggr = aggregate(mlp_cross_val)

# Print the results.
print('Dummy:', dummy_aggr)
print('Decision tree:', tree_aggr)
print('Random forest:', forest_aggr)
print('Gradient boosting:', boost_aggr)
print('Perceptron:', perceptron_aggr)
print('Logistic regression:', logreg_aggr)
print('Linear SVC:', svc_aggr)
print('MLP:', mlp_aggr)

/Users/samkaj/code/ml/appliedml-1/ass1/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/samkaj/code/ml/appliedml-1/ass1/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

Dummy: 3.902941176470588
Decision tree: 4.63235294117647
Random forest: 4.691176470588235
Gradient boosting: 4.747058823529412
Perceptron: 4.126470588235295
Logistic regression: 4.455882352941177
Linear SVC: 3.902941176470588
MLP: 4.411764705882353
